## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Bulawayo,-20.1500,28.5833,55.44,68,62,4.90,broken clouds
1,1,Poum,-20.2333,164.0167,79.23,71,100,9.28,overcast clouds
2,2,Upernavik,72.7868,-56.1549,2.28,72,100,5.50,overcast clouds
3,3,Chuy,-33.6971,-53.4616,61.86,76,98,6.58,overcast clouds
4,4,Hithadhoo,-0.6000,73.0833,83.79,76,100,19.95,overcast clouds
5,5,Kitsuki,33.4167,131.6167,57.20,67,75,11.50,broken clouds
6,6,Khatanga,71.9667,102.5000,-27.22,95,36,0.51,scattered clouds
7,7,Sorland,67.6670,12.6934,39.15,74,100,21.32,overcast clouds
8,8,Ribeira Grande,38.5167,-28.7000,57.81,66,0,10.71,clear sky
9,9,Barrow,71.2906,-156.7887,-4.00,59,1,23.02,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Bulawayo,-20.1500,28.5833,55.44,68,62,4.90,broken clouds
1,1,Poum,-20.2333,164.0167,79.23,71,100,9.28,overcast clouds
3,3,Chuy,-33.6971,-53.4616,61.86,76,98,6.58,overcast clouds
4,4,Hithadhoo,-0.6000,73.0833,83.79,76,100,19.95,overcast clouds
5,5,Kitsuki,33.4167,131.6167,57.20,67,75,11.50,broken clouds
8,8,Ribeira Grande,38.5167,-28.7000,57.81,66,0,10.71,clear sky
10,10,Laguna,38.4210,-121.4238,80.60,17,1,4.61,clear sky
11,11,Kota Bahru,6.1333,102.2386,84.20,84,20,3.44,few clouds
13,13,Arraial Do Cabo,-22.9661,-42.0278,74.48,77,45,16.75,scattered clouds
15,15,Busselton,-33.6500,115.3333,69.01,82,100,1.99,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()


City_ID                489
City                   489
Lat                    489
Lng                    489
Max Temp               489
Humidity               489
Cloudiness             489
Wind Speed             489
Weather description    489
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()


City_ID                489
City                   489
Lat                    489
Lng                    489
Max Temp               489
Humidity               489
Cloudiness             489
Wind Speed             489
Weather description    489
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Bulawayo,55.44,broken clouds,-20.1500,28.5833,
1,Poum,79.23,overcast clouds,-20.2333,164.0167,
3,Chuy,61.86,overcast clouds,-33.6971,-53.4616,
4,Hithadhoo,83.79,overcast clouds,-0.6000,73.0833,
5,Kitsuki,57.20,broken clouds,33.4167,131.6167,
8,Ribeira Grande,57.81,clear sky,38.5167,-28.7000,
10,Laguna,80.60,clear sky,38.4210,-121.4238,
11,Kota Bahru,84.20,few clouds,6.1333,102.2386,
13,Arraial Do Cabo,74.48,scattered clouds,-22.9661,-42.0278,
15,Busselton,69.01,overcast clouds,-33.6500,115.3333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Bulawayo,55.44,broken clouds,-20.1500,28.5833,4 On Housman Backpackers
1,Poum,79.23,overcast clouds,-20.2333,164.0167,Gîte kajeon
3,Chuy,61.86,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
4,Hithadhoo,83.79,overcast clouds,-0.6000,73.0833,Scoop Guest House
5,Kitsuki,57.20,broken clouds,33.4167,131.6167,里
...,...,...,...,...,...,...
676,Abu Samrah,48.99,few clouds,35.3029,37.1841,
677,Swinoujscie,48.99,clear sky,53.9105,14.2471,Hotel Interferie Medical SPA
679,Wheatley,50.00,overcast clouds,51.7473,-1.1394,Bat & Ball Cuddesdon
680,Puerto Madryn,59.22,overcast clouds,-42.7692,-65.0385,Hotel Tolosa


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))